## Installing transformer-rankers and dependencies

---



In [ ]:
!pip install git+https://github.com/Guzpenha/transformer_rankers.git
!wget https://raw.githubusercontent.com/Guzpenha/transformer_rankers/master/requirements.txt
!pip install -r requirements.txt

  Cloning https://github.com/Guzpenha/transformer_rankers.git to /tmp/pip-req-build-_7lci577
  Running command git clone -q https://github.com/Guzpenha/transformer_rankers.git /tmp/pip-req-build-_7lci577
  Created wheel for transformer-rankers: filename=transformer_rankers-0.0.1-cp36-none-any.whl size=49634 sha256=3b64a324804a8e4de1214835eb56822d920fa61cea283408cb7eac6a7bbbd14c
  Stored in directory: /tmp/pip-ephem-wheel-cache-eyb3kofw/wheels/8d/17/81/c0402e419a193812e5b02d153d777fcae6cfcecf519e3884cb
Successfully built transformer-rankers
--2020-12-15 18:31:53--  https://raw.githubusercontent.com/Guzpenha/transformer_rankers/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239 [text/plain]
Saving to: ‘requirements.txt.2’

requirements.txt.

In [ ]:
#Install Anserini which is also a requirement for part of transformer-rankers (BM25 Negative Samplers)
!apt-get install maven -qq
!git clone --recurse-submodules https://github.com/castorini/anserini.git
!cd anserini; mvn clean package appassembler:assemble -DskipTests -Dmaven.javadoc.skip=true
!ls anserini/target/appassembler/bin/

fatal: destination path 'anserini' already exists and is not an empty directory.
[INFO] Scanning for projects...
[INFO] 
[INFO] ------------------------< io.anserini:anserini >------------------------
[INFO] Building Anserini 0.10.1-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-clean-plugin:2.5:clean (default-clean) @ anserini ---
[INFO] Deleting /content/anserini/target
[INFO] 
[INFO] --- jacoco-maven-plugin:0.8.2:prepare-agent (default) @ anserini ---
[INFO] argLine set to -javaagent:/root/.m2/repository/org/jacoco/org.jacoco.agent/0.8.2/org.jacoco.agent-0.8.2-runtime.jar=destfile=/content/anserini/target/jacoco.exec
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ anserini ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 233 resources
[INFO] 
[INFO] --- maven-compiler-plugin:3.8.1:compile (default-compile) @ anserini ---
[INFO] Changes detected - recompiling t

In [ ]:
#Google colab with torch 1.5 doesnt see the GPU
!pip install -I torch==1.4.0
import torch
torch.cuda.get_device_name(0)  #This should ouptut a GPU device name

  Using cached https://files.pythonhosted.org/packages/24/19/4804aea17cd136f1705a5e98a00618cb8f6ccc375ad8bfa437408e09d058/torch-1.4.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.4.0 which is incompatible.


'Tesla P4'

## Downloading ClariQ data

In [ ]:
!mkdir data
!mkdir data/clariq
!cd data/clariq; wget https://github.com/aliannejadi/ClariQ/raw/master/data/dev.tsv
!cd data/clariq; wget https://github.com/aliannejadi/ClariQ/raw/master/data/train.tsv
!cd data/clariq; wget https://github.com/aliannejadi/ClariQ/raw/master/data/question_bank.tsv
!mv data/clariq/train.tsv data/clariq/train_original.tsv

In [ ]:
!rm -rf data/clariq/anserini_train

## Preprocess ClariQ for transformer-rankers

In [ ]:
import pandas as pd
data_path = "./data/"

train = pd.read_csv(data_path+"clariq/train_original.tsv", sep="\t")
valid = pd.read_csv(data_path+"clariq/dev.tsv", sep="\t")

train = train[["initial_request", "question"]]
train.columns = ["query", "clarifying_question"]
train = train[~train["clarifying_question"].isnull()]

valid = valid[["initial_request", "question"]]
valid.columns = ["query", "clarifying_question"]
valid = valid[~valid["clarifying_question"].isnull()]

train.to_csv(data_path+"clariq/train.tsv", sep="\t", index=False)
valid.to_csv(data_path+"clariq/valid.tsv", sep="\t", index=False)

In [ ]:
# For transformer-rankers we only need a pandas DF with query (here the initial request) 
# and relevant documents (here the clarifying questions).
train.head()

,query,clarifying_question
0,Tell me about Obama family tree.,are you interested in seeing barack obamas family
1,Tell me about Obama family tree.,would you like to know barack obamas geneology
2,Tell me about Obama family tree.,would you like to know about obamas ancestors
3,Tell me about Obama family tree.,would you like to know who is currently alive ...
4,Tell me about Obama family tree.,are you looking for biological information on ...


In [ ]:
# We will sample negative samples for training using the question bank
question_bank = pd.read_csv(data_path+"clariq/question_bank.tsv", sep="\t")
question_bank.head()

,question_id,question
0,Q00001,NaN
1,Q00002,a total cholesterol of 180 to 200 mgdl 10 to 1...
2,Q00003,about how many years experience do you want th...
3,Q00004,according to anima the bible or what other source
4,Q00005,ae you looking for examples of septic system d...


In [ ]:
import wandb
wandb.init()

wandb: Wandb version 0.10.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


2020-12-15 18:36:48,673 [INFO] system metrics and metadata threads started
2020-12-15 18:36:48,674 [INFO] checking resume status, waiting at most 10 seconds
2020-12-15 18:36:48,731 [INFO] resuming run from id: UnVuOnYxOjI4cTNyN2VhOnVuY2F0ZWdvcml6ZWQ6Z3V6
2020-12-15 18:36:48,744 [INFO] upserting run before process can begin, waiting at most 10 seconds
2020-12-15 18:36:48,799 [INFO] saving patches
2020-12-15 18:36:48,800 [INFO] saving pip packages
2020-12-15 18:36:48,803 [INFO] initializing streaming files api
2020-12-15 18:36:48,809 [INFO] unblocking file change observer, beginning sync with W&B servers


W&B Run: https://app.wandb.ai/guz/uncategorized/runs/28q3r7ea

2020-12-15 18:36:48,828 [INFO] shutting down system stats and metadata service
2020-12-15 18:36:49,394 [INFO] file/dir modified: /content/wandb/run-20201215_183648-28q3r7ea/config.yaml
2020-12-15 18:36:49,466 [INFO] file/dir created: /content/wandb/run-20201215_183648-28q3r7ea/wandb-history.jsonl
2020-12-15 18:36:49,468 [INFO] file/dir created: /content/wandb/run-20201215_183648-28q3r7ea/requirements.txt
2020-12-15 18:36:49,476 [INFO] file/dir created: /content/wandb/run-20201215_183648-28q3r7ea/wandb-metadata.json
2020-12-15 18:36:49,487 [INFO] file/dir created: /content/wandb/run-20201215_183648-28q3r7ea/wandb-summary.json
2020-12-15 18:36:49,490 [INFO] file/dir created: /content/wandb/run-20201215_183648-28q3r7ea/wandb-events.jsonl
2020-12-15 18:36:49,502 [INFO] file/dir created: /content/wandb/run-20201215_183648-28q3r7ea/code
2020-12-15 18:36:49,675 [INFO] stopping streaming files and file change observer
2020-12-15 18:36:50,394 [INFO] file/dir modified: /content/wandb/run-2020121

## Training a transformer-ranker for ClariQ (RQ2)

The problem is to retrieve the most relevant clarifying question for a given query. We will train a BERT-ranker using transformer-rankers.

In [ ]:
from transformer_rankers.trainers import transformer_trainer
from transformer_rankers.datasets import dataset
from transformer_rankers.negative_samplers import negative_sampling
from transformer_rankers.eval import results_analyses_tools

from transformers import BertTokenizer, BertForSequenceClassification

import logging
import os
import sys
import torch
import random
import numpy as np

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

logging.basicConfig(
  level=logging.INFO,
  format="%(asctime)s [%(levelname)s] %(message)s",
  handlers=[
      logging.StreamHandler(sys.stdout)
  ]
)

#The combination of query and question are not that big.
max_seq_len = 50

#Lets use an almost balanced amount of positive and negative samples during training.
average_relevant_per_query = train.groupby("query").count().mean().values[0]

#Instantiate BM25 negative sampler.
# ns_train = negative_sampling.BM25NegativeSamplerPyserini(list(question_bank["question"].values[1:]), 9 , 
#                     "/content/data/clariq/anserini_train/", -1, "./anserini/")
# ns_val = negative_sampling.BM25NegativeSamplerPyserini(list(question_bank["question"].values[1:]), 9, 
#                     "/content/data/clariq/anserini_train/", -1, "./anserini/")

# We could also use random sampling which does not require Anserini.
ns_train = negative_sampling.RandomNegativeSampler(list(question_bank["question"].values[1:]), int(average_relevant_per_query))
ns_val = negative_sampling.RandomNegativeSampler(list(question_bank["question"].values[1:]), int(average_relevant_per_query))

#Create the loaders for the dataset, with the respective negative samplers
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
dataloader = dataset.QueryDocumentDataLoader(train_df=train,
                    val_df=valid, test_df=valid,
                    tokenizer=tokenizer, negative_sampler_train=ns_train,
                    negative_sampler_val=ns_val, task_type='classification',
                    train_batch_size=8, val_batch_size=8, max_seq_len=max_seq_len,
                    sample_data=-1, cache_path="./data/clariq/")

train_loader, val_loader, test_loader = dataloader.\
  get_pytorch_dataloaders()

#Use BERT (any model that has SequenceClassification class from HuggingFace would work here)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

#Instantiate trainer that handles fitting.
trainer = transformer_trainer.TransformerTrainer(model=model,
  train_loader=train_loader,
  val_loader=val_loader, test_loader=test_loader,
  num_ns_eval=int(average_relevant_per_query), task_type="classification", tokenizer=tokenizer,
  validate_every_epochs=1, num_validation_batches=-1,
  num_epochs=1, lr=5e-7, sacred_ex=None)

#Train (our validation eval uses the NS sampling procedure)
trainer.fit()

2020-12-15 18:36:59,320 [INFO] Train instances per batch 8
2020-12-15 18:36:59,361 [INFO] Generating instances with signature pointwise_set_train_n_cand_docs_45_ns_sampler_RandomNS_seq_max_l_50_sample_-1_for_classification_using_BertTokenizer


100%|██████████| 187/187 [00:00<00:00, 7287.93it/s]

2020-12-15 18:36:59,392 [INFO] Encoding examples using tokenizer.batch_encode_plus().


2020-12-15 18:37:05,937 [INFO] Transforming examples to instances format.
2020-12-15 18:37:05,997 [INFO] Set train Instance 0 query 

Child support in Indiana?[...]

2020-12-15 18:37:05,998 [INFO] Set train Instance 0 document 

are you interested in indiana child support

2020-12-15 18:37:05,999 [INFO] Set train Instance 0 features 

InputFeatures(input_ids=[101, 2775, 2490, 1999, 5242, 1029, 102, 2024, 2017, 4699, 1999, 5242, 2775, 2490, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label=1)

2020-12-15 18:37:06,000 [INFO] Set train Instance 1 query 

Child support in Indiana?[...]

2020-12-1

## Evaluating with ClariQ evaluation scripts
The above code uses the transformer-ranker's built-in evaluation. This means that we are only ranking from a set of K (int(average_relevant_per_query) in the example) candidate questions, a re-ranking scenario where all the positive examples are in the candidate list. RQ2 of ClariQ requires us to rank from the entire question_bank. Additionally it evaluates whether the clarifying questions helps for document retrieval.

In [ ]:
! git clone https://github.com/aliannejadi/ClariQ.git ClariQ-repo
! pip install rank_bm25

### Re-rank BM25 with the fine-tuned BERT-ranker

Lets first use the bm25 [example](https://colab.research.google.com/drive/1g_Sc9j5fYT1hiOxif6BVH5NHNt-icxtT#scrollTo=_7_2LTXoXqK7) from Mohammad to generate the dataset.

In [ ]:
rerank_top_k = 30

In [ ]:
# Imports required packages, defines stem & tokenizez function
import pandas as pd
from rank_bm25 import BM25Okapi
import nltk
from nltk.stem.porter import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

def stem_tokenize(text, remove_stopwords=True):
  stemmer = PorterStemmer()
  tokens = [word for sent in nltk.sent_tokenize(text) \
                                      for word in nltk.word_tokenize(sent)]
  tokens = [word for word in tokens if word not in \
          nltk.corpus.stopwords.words('english')]
  return [stemmer.stem(word) for word in tokens]

# Files paths
request_file_path = './ClariQ-repo/data/dev.tsv'
question_bank_path = './ClariQ-repo/data/question_bank.tsv'
run_file_path = './ClariQ-repo/sample_runs/dev_bm25'

# Reads files and build bm25 corpus (index)
dev = pd.read_csv(request_file_path, sep='\t')
question_bank = pd.read_csv(question_bank_path, sep='\t').fillna('')
question_bank['tokenized_question_list'] = question_bank['question'].map(stem_tokenize)
question_bank['tokenized_question_str'] = question_bank['tokenized_question_list'].map(lambda x: ' '.join(x))
bm25_corpus = question_bank['tokenized_question_list'].tolist()
bm25 = BM25Okapi(bm25_corpus)

# Runs bm25 for every query and stores output in file.
examples = []
all_preds_bm25 = []
with open(run_file_path, 'w') as fo:
  for tid in dev['topic_id'].unique():
    query = dev.loc[dev['topic_id']==tid, 'initial_request'].tolist()[0]
    bm25_ranked_list = bm25.get_top_n(stem_tokenize(query, True), 
                                    bm25_corpus, 
                                    n=rerank_top_k)
    bm25_q_list = [' '.join(sent) for sent in bm25_ranked_list]
    docs = question_bank.set_index('tokenized_question_str').loc[bm25_q_list, 'question'].tolist()
    preds = question_bank.set_index('tokenized_question_str').loc[bm25_q_list, 'question_id'].tolist()
    all_preds_bm25.append(preds)
    for doc in docs[:rerank_top_k]:
      examples.append((query, doc))

Now we are going to transform this dataset in the format required for our model.

In [ ]:
from transformers.data.data_collator import default_data_collator
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset, DataLoader
from transformer_rankers.utils import utils

class SimpleDataset(Dataset):
    def __init__(self, features):
        self.features = features
    def __len__(self):
        return len(self.features)
    def __getitem__(self, index):
        return self.features[index]

batch_encoding = tokenizer.batch_encode_plus(examples, 
                max_length=max_seq_len, pad_to_max_length=True)
features = []
for i in range(len(examples)):
    inputs = {k: batch_encoding[k][i] for k in batch_encoding}
    feature = InputFeatures(**inputs, label=0)
    features.append(feature)

dataset = SimpleDataset(features)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False, collate_fn=default_data_collator)

Now we can run the trained model on this dataset and  save the predictions to a file

In [ ]:
logits, _, softmax_output = trainer.predict(dataloader)
softmax_output_by_query = utils.acumulate_list(softmax_output[0], rerank_top_k)

In [ ]:
import numpy as np
run_file_path = './ClariQ-repo/sample_runs/dev_BERT-reranker'
with open(run_file_path, 'w') as fo:
  for tid_idx, tid in enumerate(dev['topic_id'].unique()):
    document_scores = np.array(softmax_output_by_query[tid_idx])
    top_k_scores_idx = (-document_scores).argsort()[:rerank_top_k]  
    preds = np.array(all_preds_bm25[tid_idx])[top_k_scores_idx]
    for i, qid in enumerate(preds):
      fo.write('{} 0 {} {} {} BERT-reranker\n'.format(tid, qid, i, len(preds)-i))

In [ ]:
# Report question relevance performance
! python ./ClariQ-repo/src/clariq_eval_tool.py  --eval_task question_relevance\
                                                --data_dir ./ClariQ-repo/data/ \
                                                --experiment_type dev \
                                                --run_file {run_file_path} \
                                                --out_file {run_file_path}_question_relevance.eval

In [ ]:
! python ./ClariQ-repo/src/clariq_eval_tool.py  --eval_task document_relevance\
                                                --data_dir ./ClariQ-repo/data/ \
                                                --experiment_type dev \
                                                --run_file {run_file_path} \
                                                --out_file {run_file_path}.eval

### Full retrieval
So let's first generate a dataset containing all combinations of dev queries 
and question_bank questions.

In [ ]:
from transformers.data.data_collator import default_data_collator
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset, DataLoader

class SimpleDataset(Dataset):
    def __init__(self, features):
        self.features = features
    def __len__(self):
        return len(self.features)
    def __getitem__(self, index):
        return self.features[index]

#Lets not use the null document for no question.
all_documents = list(question_bank["question"].values[1:])
examples = []
for tid in dev['topic_id'].unique():
    query = dev.loc[dev['topic_id']==tid, 'initial_request'].tolist()[0]
    for doc in all_documents:
      examples.append((query, doc))

batch_encoding = tokenizer.batch_encode_plus(examples, 
                max_length=max_seq_len, pad_to_max_length=True)
features = []
for i in range(len(examples)):
    inputs = {k: batch_encoding[k][i] for k in batch_encoding}
    feature = InputFeatures(**inputs, label=0)
    features.append(feature)

dataset = SimpleDataset(features)

Now we have to make the predictions and acumulate the logits by the number of candidate documents

In [ ]:
dataloader = DataLoader(dataset, batch_size=16, shuffle=False, collate_fn=default_data_collator)
from transformer_rankers.utils import utils
logits, _, softmax_output = trainer.predict(dataloader)
softmax_output_by_query = utils.acumulate_list(softmax_output[0], len(all_documents))

In [ ]:
import numpy as np
run_file_path = './ClariQ-repo/sample_runs/dev_BERT-ranker'
all_doc_ids = np.array(question_bank["question_id"].values[1:])
with open(run_file_path, 'w') as fo:
  for tid_idx, tid in enumerate(dev['topic_id'].unique()):
    all_documents_scores = np.array(softmax_output_by_query[tid_idx])
    top_30_scores_idx = (-all_documents_scores).argsort()[:30]  
    preds = all_doc_ids[top_30_scores_idx]
    for i, qid in enumerate(preds):    
      fo.write('{} 0 {} {} {} BERT-ranker\n'.format(tid, qid, i, len(preds)-i))

In [ ]:
# Report question relevance performance
! python ./ClariQ-repo/src/clariq_eval_tool.py  --eval_task question_relevance\
                                                --data_dir ./ClariQ-repo/data/ \
                                                --experiment_type dev \
                                                --run_file {run_file_path} \
                                                --out_file {run_file_path}_question_relevance.eval

In [ ]:
! python ./ClariQ-repo/src/clariq_eval_tool.py  --eval_task document_relevance\
                                                --data_dir ./ClariQ-repo/data/ \
                                                --experiment_type dev \
                                                --run_file {run_file_path} \
                                                --out_file {run_file_path}.eval

## Results comparison

In [ ]:
import json

models = ["bm25", "BERT-reranker", "BERT-ranker"]
results = []
for model in models:
  with open('./ClariQ-repo/sample_runs/dev_{}_question_relevance.eval'.format(model)) as f:
    res = json.load(f)
    for metric_name in res:
        metric_avg = np.mean([res[metric_name][k] for k in res[metric_name]])
        results.append([model, metric_name, metric_avg])
res_df = pd.DataFrame(results, columns = ["model", "metric", "value"])

pd.set_option("display.precision", 4)
res_df = res_df.set_index(["model", "metric"]).unstack()
cols = res_df.columns.tolist()
res_df.sort_values([("value","Recall10")])[cols[-1:] + cols[:-1]]

In [ ]:
import json

models = ["bm25", "BERT-reranker", "BERT-ranker"]
results = []
for model in models:
  with open('./ClariQ-repo/sample_runs/dev_{}.eval'.format(model)) as f:
    res = json.load(f)
    for metric_name in res:
        metric_avg = np.mean([res[metric_name][k] for k in res[metric_name]])
        results.append([model, metric_name, metric_avg])
res_df = pd.DataFrame(results, columns = ["model", "metric", "value"])

pd.set_option("display.precision", 4)
res_df = res_df.set_index(["model", "metric"]).unstack()
res_df.sort_values(("value", "MRR100"))